In [1]:
import random
import argparse
import numpy as np
import datetime as dt

import torch
import torchvision
import avalanche
import torch.backends.cudnn as cudnn

from avalanche.evaluation import metrics
from avalanche.logging import InteractiveLogger, TensorboardLogger

from avalanche.training.plugins import ReplayPlugin, EvaluationPlugin
from avalanche.training.storage_policy import ClassBalancedBuffer
from avalanche.training.storage_policy import ReservoirSamplingBuffer

### Argument

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=0)
parser.add_argument('--num_class', type=int, default=10)
parser.add_argument('--dataset', type=str, default='tinyImage', choices=['cifar10', 'cifar10', 'tinyImage'])
parser.add_argument('--device', type=str, default='0')
parser.add_argument('--lr', '--learning_rate', type=float, default=0.001)
parser.add_argument('--train_mb', type=int, default=512)
parser.add_argument('--eval_mb', type=int, default=256)
parser.add_argument('--epoch', type=int, default=70)
parser.add_argument('--dataset_root', type=str, default='./dataset/')
# continual learning settings
parser.add_argument('--increment', type=int, default=10)
parser.add_argument('--memory_size', type=int, default=500)
parser.add_argument('--buffer_weights', type=float, default=0, help='random uniform value in [0, 1]')
# LwF settings
parser.add_argument('--alpha', type=float, default=1.)
parser.add_argument('--temperature', type=float, default=2.)

args = parser.parse_args(args=[])

### Set Seed & Device

In [3]:
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
cudnn.enabled = False
cudnn.deterministic = True

device = torch.device('cuda:' + args.device if torch.cuda.is_available() else 'cpu')
args.device = torch.device(device)

### Load Dataset

In [4]:
print('Dataset : ', args.dataset)

if args.dataset == 'cifar10':
    #cifar10 train and eval used default transform (default)
    benchmark = avalanche.benchmarks.SplitCIFAR10(n_experiences=args.increment,
                                                  return_task_id=False,
                                                  seed=args.seed,
                                                  shuffle=True,
                                                  dataset_root=args.dataset_root)
elif args.dataset == 'cifar100':
    #cifar100 train and eval used default transform
    args.num_class = 100
    benchmark = avalanche.benchmarks.SplitCIFAR100(n_experiences=args.increment,
                                                  return_task_id=False,
                                                  seed=args.seed,
                                                  shuffle=True,
                                                  dataset_root=args.dataset_root)
    
elif args.dataset == 'tinyImage':
    args.num_class = 200
    benchmark = avalanche.benchmarks.SplitTinyImageNet(n_experiences=args.increment,
                                                       return_task_id=True,
                                                       seed=args.seed,
                                                       shuffle=True,
                                                       dataset_root=args.dataset_root)

Dataset :  tinyImage
Files already downloaded and verified
Files already downloaded and verified


### Evaulation

In [5]:
date = dt.datetime.now()
date = date.strftime("%Y_%m_%d_%H_%M_%S")

interactive_logger = InteractiveLogger()
tensor_logger = TensorboardLogger("logs/" + args.dataset + '/' + date)

eval_plugin = EvaluationPlugin(
    metrics.accuracy_metrics(epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, tensor_logger]
)

### Backbone Model

In [6]:
model = torchvision.models.resnet18(pretrained=False, num_classes=args.num_class)
model.to(args.device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)

# storage_policy = ClassBalancedBuffer(args.memory_size, adaptive_size=True)
storage_policy = ReservoirSamplingBuffer(max_size=args.memory_size) # init = 0
replay_plugin = ReplayPlugin(args.memory_size, storage_policy=storage_policy)

/home/heonsung/miniconda3/envs/lab/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/heonsung/miniconda3/envs/lab/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [7]:
cl_strategy = avalanche.training.LwF(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    alpha=args.alpha,
    temperature=args.temperature,
    train_mb_size=args.train_mb,
    train_epochs=args.epoch,
    eval_mb_size=args.eval_mb,
    device=args.device,
    plugins=[replay_plugin],
    evaluator=eval_plugin)

In [8]:
res = None
for experience in benchmark.train_stream:
    cl_strategy.train(experience)
    res = cl_strategy.eval(benchmark.test_stream)

-- >> Start of training phase << --
100%|██████████| 20/20 [00:15<00:00,  1.30it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0204
100%|██████████| 20/20 [00:14<00:00,  1.36it/s]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0423
100%|██████████| 20/20 [00:14<00:00,  1.36it/s]
Epoch 2 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0528
100%|██████████| 20/20 [00:14<00:00,  1.36it/s]
Epoch 3 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0613
100%|██████████| 20/20 [00:14<00:00,  1.35it/s]
Epoch 4 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0635
100%|██████████| 20/20 [00:14<00:00,  1.34it/s]
Epoch 5 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0715
100%|██████████| 20/20 [00:14<00:00,  1.34it/s]
Epoch 6 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0865
100%|██████████| 20/20 [00:14<00:00,  1.33it/s]
Epoch 7 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0